In [1]:
from pathlib import Path
import tensorflow as tf
from utils import download, extract, get_cifar10_batches

In [2]:
cifar_10_download = Path("data/cifar-10-python.tar.gz")
if not cifar_10_download.exists():
    download("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", cifar_10_download)

In [3]:
extract_loc = Path("data/cifar-10-batches-py")
if not extract_loc.exists():
    extract(cifar_10_download)

In [4]:
batches = get_cifar10_batches()

In [6]:
batch_size = 16
n = 32
n_channels = 3

In [8]:
inp = tf.placeholder(tf.float32, [batch_size, n, n, n_channels])
category = tf.placeholder(tf.int32, [batch_size, 1])